(graph-example)=
# Advanced model serving graph - notebook example

This example demonstrates how to use MLRun serving graphs and their advanced functionality including:
* Use of flow, task, model, and ensemble router states
* Build tasks from custom handlers, classes and storey components
* Use custom error handlers
* Test graphs locally
* Deploy the graph as a real-time serverless functions

**In this example**
- [Define functions and classes used in the graph](#define-functions-and-classes-used-in-the-graph)
- [Create a new serving function and graph](#create-a-new-serving-function-and-graph)
- [Test the function locally](#test-the-function-locally)
- [Deploy the graph as a real-time serverless function](#deploy-the-graph-as-a-real-time-serverless-function)

## Define functions and classes used in the graph

In [2]:
from cloudpickle import load
from typing import List
from sklearn.datasets import load_iris
import numpy as np


# model serving class example
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model(".pkl")
        self.model = load(open(model_file, "rb"))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body["inputs"])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()


# echo class, custom class example
class Echo:
    def __init__(self, context, name=None, **kw):
        self.context = context
        self.name = name
        self.kw = kw

    def do(self, x):
        print("Echo:", self.name, x)
        return x


# error echo function, demo catching error and using custom function
def error_catcher(x):
    x.body = {"body": x.body, "origin_state": x.origin_state, "error": x.error}
    print("EchoError:", x)
    return None

In [3]:
# mark the end of the code section, DO NOT REMOVE !
# mlrun: end-code

## Create a new serving function and graph
Use `code_to_function` to convert the above code into a serving function object and initialize a graph with async flow topology.

In [4]:
function = mlrun.code_to_function(
    "advanced", kind="serving", image="mlrun/mlrun", requirements=["storey"]
)
graph = function.set_topology("flow", engine="async")
# function.verbose = True

Specify the sklearn models that are used in the ensemble.

In [5]:
models_path = "https://s3.wasabisys.com/iguazio/models/iris/model.pkl"
path1 = models_path
path2 = models_path

Build and connect the graph (DAG) using the custom function and classes and plot the result. Add states 
using the `state.to()` method (adds a new state after the current one), or using the 
`graph.add_step()` method.

Use the graph `error_handler` if you want an error from the graph or a step to be fed into a specific state (catcher). See the full description in {ref}`pipelines-error-handling`.

You can specify which state is the responder (returns the HTTP response) using the `state.respond()` method.
If you don't specify the responder, the graph is non-blocking.

In [6]:
# use built-in storey class or our custom Echo class to create and link Task steps. Add an error handling step that runs if the grah step fails
graph.to("storey.Extend", name="enrich", _fn='({"tag": "something"})').to(
    class_name="Echo", name="pre-process", some_arg="abc"
).error_handler(name="catcher", handler="handle_error", full_event=True)

# add an Ensemble router with two child models (routes). The "*" prefix mark it is a router class
router = graph.add_step(
    "*mlrun.serving.VotingEnsemble", name="ensemble", after="pre-process"
)
router.add_route("m1", class_name="ClassifierModel", model_path=path1)
router.add_route("m2", class_name="ClassifierModel", model_path=path2)

# add the final step (after the router) that handles post processing and responds to the client
graph.add_step(class_name="Echo", name="final", after="ensemble").respond()

# plot the graph (using Graphviz) and run a test
graph.plot(rankdir="LR")

## Test the function locally
Create a test set.

In [7]:
import random

iris = load_iris()
x = random.sample(iris["data"].tolist(), 5)

Create a mock server (simulator) and test the graph with the test data.

> Note: The model and router objects support a common serving protocol API, see the [protocol and API section](model-api.html).

In [8]:
server = function.to_mock_server()
resp = server.test("/v2/models/infer", body={"inputs": x})
server.wait_for_completion()
resp

## Deploy the graph as a real-time serverless function

In [9]:
function.deploy()

<b>Invoke the remote function using the test data</b>

In [12]:
function.invoke("/v2/models/infer", body={"inputs": x})